In [56]:
# !pip install mysql-connector-python-rf
# !pip install mysqlclient

In [61]:
import mysql.connector
import pandas as pd 
import json
import numpy as np
import MySQLdb
import MySQLdb.cursors

In [62]:
db_thread = MySQLdb.connect("127.0.0.1", "root", "", "testing2", cursorclass=MySQLdb.cursors.DictCursor)
cur_thread = db_thread.cursor()
print(cur_thread)

In [63]:
cur_thread.execute("SELECT * FROM paket u WHERE STATUS=1;")
pakets = cur_thread.fetchall()
for paket in pakets:
    sql = "SELECT * FROM soal where paket_id='%s'" % (paket['id'])
    soal = pd.read_sql(sql, con=db_thread)
    soal = soal.set_index('id')
    soal['salah_bawah'] = 0
    soal['salah_atas'] = 0
    
    sql = "select count(*) as jumlah from ujian_siswa where paket_id='%s' and status=10" % (paket['id'])
    cur_thread.execute(sql)
    ujian_siswas = cur_thread.fetchall()
    batas_jumlah = int(ujian_siswas[0]['jumlah']*0.27)
    print(batas_jumlah)
    
    #nilai tinggi
    sql = "select * from ujian_siswa u WHERE u.paket_id='%s' AND u.status=10 ORDER BY u.jumlah_benar DESC LIMIT %d" %(paket['id'], batas_jumlah)
    cur_thread.execute(sql)
    ujian_siswas_atas = cur_thread.fetchall()
    
    #nilai_rendah
    sql = "select * from ujian_siswa u WHERE u.paket_id='%s' AND u.status=10 ORDER BY u.jumlah_benar ASC LIMIT %d" %(paket['id'], batas_jumlah)
    cur_thread.execute(sql)
    ujian_siswas_bawah = cur_thread.fetchall()
    
    for ujian_siswa in ujian_siswas_atas:
        print("ujian siswa id: ", ujian_siswa['id'])
        soals = json.loads(ujian_siswa["random_soal"])
        for soal_id in soals:
            benar_siswa = json.loads(soal.loc[soal_id, 'siswa_id_benar'])
            if(soal.loc[soal_id, 'tipe_soal'] == 'pilihan_ganda'):
                if not(ujian_siswa['id'] in benar_siswa): 
                    soal.loc[soal_id, 'salah_atas'] += 1
                    
    for ujian_siswa in ujian_siswas_bawah:
        print("ujian siswa id: ", ujian_siswa['id'])
        soals = json.loads(ujian_siswa["random_soal"])
        for soal_id in soals:
            benar_siswa = json.loads(soal.loc[soal_id, 'siswa_id_benar'])
            if(soal.loc[soal_id, 'tipe_soal'] == 'pilihan_ganda'):
                if not(ujian_siswa['id'] in benar_siswa): 
                    soal.loc[soal_id, 'salah_bawah'] += 1

    for index, soal in soal.iterrows():
        tingkat_kesulitan = (soal['salah_bawah']+soal['salah_atas'])/(batas_jumlah*2) 
        print(tingkat_kesulitan, type(tingkat_kesulitan))
        sql = "update soal_analisis set tingkat_kesulitan=%f, salah_atas=%d, salah_bawah=%d where id='%s'" % (tingkat_kesulitan, soal['salah_atas'], soal['salah_bawah'], index)
        cur_thread.execute(sql)
    db_thread.commit()
    
    print("~DONE~ paket:", paket['nama'])

663
ujian siswa id:  7286e4c0-5c2c-11ea-a8be-a13ece9b627c
ujian siswa id:  a14c8730-5c30-11ea-bbf6-dfa40664b16c
ujian siswa id:  2fc5a5d0-5c37-11ea-9291-61e67bae627b
ujian siswa id:  b9a2e6e0-5c2c-11ea-b9f8-81e0e972c0c0
ujian siswa id:  2d3da6b0-5c2c-11ea-8ac7-19cceb593f99
ujian siswa id:  300500e0-5c30-11ea-b3be-5947bc277a81
ujian siswa id:  584385b0-5c31-11ea-8739-3b684aec989c
ujian siswa id:  662b3eb0-5c2c-11ea-b993-35b300e18646
ujian siswa id:  79f4ae30-5c39-11ea-93bc-91d1b84c38ed
ujian siswa id:  a2cc31f0-5c2d-11ea-8a4a-058b9168b7b9
ujian siswa id:  cf4a3af0-5c2b-11ea-955d-f51d72eb9811
ujian siswa id:  dfa837d0-5c2f-11ea-838b-a5ce9c2a801d
ujian siswa id:  1cdbc690-5c2d-11ea-84e8-a96a01831e79
ujian siswa id:  313de6a0-5c2c-11ea-b557-1b2cc54b3015
ujian siswa id:  6df6f760-5c2d-11ea-836b-b79b9ec5c7bc
ujian siswa id:  701ebf30-5c30-11ea-b6f5-a94120cdb1f8
ujian siswa id:  b14cce80-5c2e-11ea-9123-51b22367adcb
ujian siswa id:  0c970aa0-5c31-11ea-8a6d-2fe739cbddac
ujian siswa id:  46f43f0

ujian siswa id:  8474d2f0-5c2d-11ea-9cfd-970847c875f3
ujian siswa id:  8c0991d0-5c2d-11ea-a9f3-8918a3efaa36
ujian siswa id:  961ad510-5c2d-11ea-b681-95e9bb7c274e
ujian siswa id:  9671fce0-5c2d-11ea-8f9e-f13fa7f1b363
ujian siswa id:  9e1d47d0-5c30-11ea-9d87-3977b47f71a8
ujian siswa id:  9e26ad00-5c2c-11ea-b895-97dd6285fb51
ujian siswa id:  a18348e0-5c2f-11ea-a61d-4fcbb47a0d1d
ujian siswa id:  b2051d60-5c2e-11ea-bbcb-2da8e883af54
ujian siswa id:  b93df0b0-5c32-11ea-8b99-df321407b7ad
ujian siswa id:  b9430420-5c2c-11ea-9c6b-bb36dfb594b6
ujian siswa id:  ba828ce0-5c2d-11ea-9f15-393505e6e33c
ujian siswa id:  cfc8ef70-5c2e-11ea-a416-9f8b1fa842e4
ujian siswa id:  d322bae0-5c2e-11ea-b27a-fdcb6bcb7f36
ujian siswa id:  d4a77ae0-5c2e-11ea-a3bc-03991eba5279
ujian siswa id:  d71e1ec0-5c2c-11ea-b0c2-1d65b2cdc26a
ujian siswa id:  dac0fcc0-5c2c-11ea-90c4-c3535895695a
ujian siswa id:  e8542840-5c2f-11ea-9a13-ed973dfc2f3b
ujian siswa id:  e9204da0-5c2d-11ea-9377-95d61c0fb927
ujian siswa id:  eca7c4c0-5c

ujian siswa id:  734eee60-5c2d-11ea-a12f-cb9321be9c82
ujian siswa id:  76fad9b0-5c35-11ea-989e-89ab7d285967
ujian siswa id:  7960b6c0-5c30-11ea-b58a-51a0ebcb42a6
ujian siswa id:  79f1e630-5c2f-11ea-8108-bd4a27cb47f0
ujian siswa id:  7c0133e0-5c2c-11ea-9d96-412d7c9bb281
ujian siswa id:  7cdffe00-5c35-11ea-83ce-73a5fae5921a
ujian siswa id:  7dbdc6b0-5c30-11ea-80f2-0d15d001b31a
ujian siswa id:  7e4c77a0-5c30-11ea-867d-2d1ff2ccbc0b
ujian siswa id:  8031a050-5c2f-11ea-b9c8-0d12ca79bb0a
ujian siswa id:  823ea7b0-5c2c-11ea-81b0-918a74e843b0
ujian siswa id:  892a7c50-5c30-11ea-87ae-07a876ef46c1
ujian siswa id:  91ee8c60-5c30-11ea-984d-cf7fec39637e
ujian siswa id:  92ddd680-5c2c-11ea-a485-43cb691ae817
ujian siswa id:  94f47a60-5c32-11ea-9e3c-81391b06fca6
ujian siswa id:  988fb360-5c2d-11ea-8e7c-41cdd653b892
ujian siswa id:  9f4fc940-5c2e-11ea-8cff-3def7651d574
ujian siswa id:  a26de6c0-5c2e-11ea-b416-f7f9b327b209
ujian siswa id:  a485a520-5c32-11ea-92c9-758483f49060
ujian siswa id:  aec34fc0-5c

ujian siswa id:  32bbe350-5c33-11ea-8f9a-a57cd7c80bbd
ujian siswa id:  3375e680-5c37-11ea-af2d-33003eb9b240
ujian siswa id:  33ab30d0-5c2c-11ea-8ad8-432aa031ee39
ujian siswa id:  33d168b0-5c36-11ea-875b-b9f8dd6be2b2
ujian siswa id:  37576220-5c2d-11ea-9ba1-d3e57ed2c228
ujian siswa id:  3a5797b0-5c2d-11ea-b280-d3f59c85b234
ujian siswa id:  3bb06810-5c30-11ea-9a6d-555612190ac1
ujian siswa id:  3d71ee80-5c39-11ea-8eac-dd8347df62e8
ujian siswa id:  3dae6970-5c2c-11ea-bc74-a902d15ff60b
ujian siswa id:  41c95190-5c2d-11ea-8c6e-4be2e8982d47
ujian siswa id:  44a88340-5c2d-11ea-9203-bf3e0c4a48a0
ujian siswa id:  459f9230-5c39-11ea-9f37-f1b7d08591c7
ujian siswa id:  47016fd0-5c37-11ea-b0bf-73f0d2519ddb
ujian siswa id:  48130c20-5c2f-11ea-9822-67614eec640e
ujian siswa id:  48d49140-5c2e-11ea-90a0-2d34b490a319
ujian siswa id:  4a459280-5c2d-11ea-8f3b-bd8f210fc994
ujian siswa id:  4afd95d0-5c2e-11ea-bf44-bf282bab1a86
ujian siswa id:  4b2c6fd0-5c2c-11ea-8c1b-1be0c43bc3f3
ujian siswa id:  4bd8ae00-5c

ujian siswa id:  a925e520-5c2e-11ea-bfd5-7d3bab06dd1c
ujian siswa id:  ab3e6b90-5c32-11ea-b52e-3b89011eb7e9
ujian siswa id:  ab8b9d00-5c2e-11ea-989d-21eef986361e
ujian siswa id:  b0ae6450-5c2e-11ea-8ea9-99e4d9d987f3
ujian siswa id:  b87d30e0-5c33-11ea-b982-add1cedfd468
ujian siswa id:  b8b5f0f0-5c2e-11ea-ab59-715f22d96cd9
ujian siswa id:  b90ec2c0-5c2d-11ea-869a-b39c95a6c9b8
ujian siswa id:  ba2fb060-5c2d-11ea-b13f-abfca93da510
ujian siswa id:  bc06ba80-5c33-11ea-9b03-13e7c133e15c
ujian siswa id:  bc9b1ed0-5c2c-11ea-a920-e566ab75fb49
ujian siswa id:  bf0e5ad0-5c2e-11ea-b706-091acce59188
ujian siswa id:  bfa291d0-5c2c-11ea-b6b1-2b3fc607a6e6
ujian siswa id:  c41d9f70-5c2c-11ea-bd96-31c29728f40e
ujian siswa id:  c5d34d10-5c2b-11ea-b95f-8345dd7da7c5
ujian siswa id:  c794b970-5c2c-11ea-8091-a1e6378c7d66
ujian siswa id:  cab15de0-5c2e-11ea-8e81-c155de4dca8d
ujian siswa id:  d190e4c0-5c2b-11ea-ab17-97adfa899b51
ujian siswa id:  d2864a70-5c2c-11ea-aeb0-17069267e7f2
ujian siswa id:  daa002e0-5c

ujian siswa id:  62e01e80-5c30-11ea-85cc-d1f4dc5cbd44
ujian siswa id:  67f25cf0-5c33-11ea-bdd5-ad7c64204bd4
ujian siswa id:  68254310-5c2e-11ea-85ce-3da789bb5c7a
ujian siswa id:  6883cca0-5c2e-11ea-89ab-1390edeb8d91
ujian siswa id:  6fffbef0-5c2f-11ea-83f8-9b8d1c093b8a
ujian siswa id:  729fd540-5c33-11ea-858c-f70a729bacfa
ujian siswa id:  74550e00-5c36-11ea-a6fa-3d716d2de6e2
ujian siswa id:  7976feb0-5c2c-11ea-a6d8-5f789c7802a2
ujian siswa id:  7c5fdda0-5c34-11ea-8c0e-175d799bd4ee
ujian siswa id:  7e817380-5c35-11ea-8f0d-7b645b15d850
ujian siswa id:  7f44f450-5c2f-11ea-91f9-b30512735244
ujian siswa id:  8dab7930-5c2e-11ea-a485-dfb1fae0ecc9
ujian siswa id:  92f7b220-5c33-11ea-b859-255de9304cb0
ujian siswa id:  9a162930-5c2f-11ea-b645-73ed14dcada9
ujian siswa id:  9dc88060-5c33-11ea-bdd1-b3cd32cf895a
ujian siswa id:  a337de00-5c2f-11ea-ba02-81e6c58086ad
ujian siswa id:  a3a3a780-5c2d-11ea-88a2-6d1f67539bd1
ujian siswa id:  a5cb5d00-5c2d-11ea-bd45-376ad49650c4
ujian siswa id:  a7333fc0-5c

ujian siswa id:  e38c0e60-5c2f-11ea-aa01-0f66f3ca8f74
ujian siswa id:  e7d28d80-5c2c-11ea-a71f-1bb357c93a3e
ujian siswa id:  eabbda40-5c2f-11ea-afd1-4d0f5f9cf6c3
ujian siswa id:  eaf07890-5c3b-11ea-84cd-ed5a4078c16d
ujian siswa id:  ee51b0f0-5c2b-11ea-a5e0-b501f4ace893
ujian siswa id:  efa947d0-5c2e-11ea-8b64-05c387155ea4
ujian siswa id:  f255d310-5c2e-11ea-a80c-53ef353d9a33
ujian siswa id:  f41e2b90-5c36-11ea-be5e-cd8d8c87750e
ujian siswa id:  f4a42ee0-5c36-11ea-90fe-e351271fb331
ujian siswa id:  f69900b0-5c2d-11ea-a60a-d93d6e72157e
ujian siswa id:  f6de8260-5c36-11ea-931f-89985c8571a0
ujian siswa id:  fa78eb20-5c2c-11ea-a946-bf3fe6cf8559
ujian siswa id:  fab829a0-5c32-11ea-bf48-67c9618b42fd
ujian siswa id:  01f54280-5c2e-11ea-b079-19c4e0e88f64
ujian siswa id:  02cbbf50-5c2c-11ea-a010-59a13149af45
ujian siswa id:  02d71ae0-5c2c-11ea-b90e-ddd9e976069e
ujian siswa id:  048d08a0-5c35-11ea-8b4f-d75e5eb165e8
ujian siswa id:  04f73350-5c37-11ea-9fb5-79f64c2397b4
ujian siswa id:  09160ff0-5c

ujian siswa id:  eb05e1a0-5c2b-11ea-8544-07bb69cb9e82
ujian siswa id:  eb8a6360-5c2d-11ea-b5b5-5fd8633558fd
ujian siswa id:  f20098f0-5c2b-11ea-a7ad-cd4b6818f580
ujian siswa id:  f21a9a80-5c2f-11ea-bcd2-355c77937f3a
ujian siswa id:  f701be20-5c2c-11ea-88fe-c39f69a81b89
ujian siswa id:  f8914200-5c30-11ea-b406-6384f6e80c00
ujian siswa id:  fbd1a740-5c31-11ea-8e28-bfde728eee6d
ujian siswa id:  fd0f3220-5c32-11ea-b9e5-af8974212ba0
ujian siswa id:  fde8c5c0-5c2b-11ea-98a0-2732e9389a95
ujian siswa id:  ff818fe0-5c34-11ea-8d43-bb7d1350aadc
ujian siswa id:  fff89c70-5c2c-11ea-b003-a9e0e893f18e
ujian siswa id:  0045f8d0-5c30-11ea-a184-bbfc14db94a3
ujian siswa id:  01985930-5c2c-11ea-b1a7-cf4f4c11ed2b
ujian siswa id:  038ee9b0-5c30-11ea-804b-993d8755dd90
ujian siswa id:  04579390-5c31-11ea-841c-195f94796b94
ujian siswa id:  0516eec0-5c31-11ea-bc3f-d9c2874759f7
ujian siswa id:  05f65d40-5c34-11ea-b0ba-71915036fb58
ujian siswa id:  0a926aa0-5c2f-11ea-9e7a-691242e2e756
ujian siswa id:  0dc1c810-5c

ujian siswa id:  eb69c810-5c2b-11ea-be69-dbc824f36637
ujian siswa id:  ed1be240-5c2b-11ea-b1e8-7fdfc9579c2e
ujian siswa id:  ed1be520-5c2b-11ea-861a-0321916b0ada
ujian siswa id:  ed811960-5c2d-11ea-b981-4b1282a04ba8
ujian siswa id:  ee5dc470-5c32-11ea-86de-99ebbbad6788
ujian siswa id:  f12521e0-5c2b-11ea-8e50-c5a0b9232470
ujian siswa id:  f3355ce0-5c2e-11ea-90c4-7d800bffb90f
ujian siswa id:  f4448b80-5c2b-11ea-b04b-51114ba69b09
ujian siswa id:  f5c14760-5c2e-11ea-ad1d-8f31e6154b50
ujian siswa id:  f798aca0-5c2c-11ea-8e35-13ba79301341
ujian siswa id:  f87ddc60-5c2e-11ea-9521-7d08d2c17367
ujian siswa id:  fd2ec7f0-5c30-11ea-80ca-cda48732009c
ujian siswa id:  fe44c940-5c30-11ea-a2c9-c5d4e2d19044
ujian siswa id:  ffa85ab0-5c30-11ea-a121-a319b6a0cf46
ujian siswa id:  01e4b730-5c30-11ea-9b62-45cc46949c72
ujian siswa id:  02ea2440-5c2e-11ea-9502-35b6f7cf6ed9
ujian siswa id:  079add20-5c31-11ea-acb4-918217bc92b8
ujian siswa id:  07b76d50-5c2c-11ea-b339-190cd565cee3
ujian siswa id:  08ab2e30-5c